In [ ]:
def testing(model, classes):

    results = pd.DataFrame([[]], columns = ['Class', 'Accuracy', 'Recall', 'Precision', 'F_score'])

    model.eval()
    y_pred1 = torch.tensor([])
    y_target1 = torch.tensor([])
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for _ in range(len(classes))]
        n_class_samples = [0 for _ in range(len(classes))]
        n_class_true_positives = [0 for _ in range(len(classes))]
        n_class_preds = [0 for _ in range(len(classes))]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
             
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            y_pred1 = torch.cat((predicted, y_pred1), dim=0)
            y_target1 = torch.cat((labels, y_target1), dim=0)
            
            for i in range(len(images)):
                label = labels[i]
                pred = predicted[i]
                if label == pred:
                    n_class_correct[label] += 1
                n_class_samples[label] += 1
                n_class_preds[pred] +=1                
                n_class_true_positives[pred] += int(pred == label)
                
                

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc} %')
        recall_arr = []
        precision_arr = []
        f_score_arr = []

        for i in range(len(classes)):
            precision = n_class_true_positives[i] / n_class_preds[i]
            recall = n_class_true_positives[i] /n_class_samples[i]
            f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            recall_arr.append(round(recall, 4))
            precision_arr.append(round(precision, 4))
            f_score_arr.append(round(f_score, 4))

            results.loc[len(results.index)] = [classes[i], n_class_correct[i] / n_class_samples[i], recall, precision, f_score] 
            
            print(f'Class: {classes[i]}')
            print(f'Accuracy: {100.0 * n_class_correct[i] / n_class_samples[i]} %')
            print('---')
        mcr = MulticlassRecall(num_classes=len(classes), average=None)
        mcp = MulticlassPrecision(num_classes=len(classes), average=None)

        print("\n Recall using torchmet", mcr(y_pred1, y_target1))
        print("Recall", recall_arr)

        print("\n Precision:", mcp(y_pred1, y_target1))
        print("Precision:", precision_arr)
        print(f_score_arr)